<a href="https://colab.research.google.com/github/kingsgameBen/iShape_tools/blob/main/ishape_measure_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BodyFat-Analysis 以3種公式，驗算體脂率結果與資料集差異
1. \
女孩體脂率(15歲以下)：(1.51 x BMI) + (0.7 x age) + 1.4 \
男孩體脂率(15歲以下)：(1.51 x BMI) + (0.7 x age) - 2.2 \
女性體脂率：(1.2 x BMI) + (0.23 x age) – 5.4. \
男性體脂率：(1.2 x BMI) + (0.23 x age) – 16.2

2. \
女性體脂率：(1.39 x BMI) + (0.16 x age) – 9 \
男性體脂率：(1.39 x BMI) + (0.16 x age) – 19.34

3. \
女性體脂率：\
495 / (1.29579 - .35004 * log10(Waist + Hip - Neck) + 0.22100 * log10(Height)) - 450 \
男性體脂率：\
495 / (1.0324 - .19077 * log10(Waist - Neck) + 0.15456 * log10(Height)) - 450


# 公式一

In [ ]:
def bmi_formula_a(gender, age, height, weight):
    kg = round(float(weight)*0.45359237, 3)
    meters = round((float(height)*2.54) / 100, 3)
    bmi = round(kg / pow(meters, 2), 1)
    # print("公式一\nage:", age, "weights:", kg, "height:", meters, "m\nbmi:", bmi)
    print("公式 bmi:", bmi)
    if gender == "m":
        if age < 15:
            bfp_men = round((1.51*bmi)+(0.7*int(age))-2.2, 2)
        elif age >= 15:
            bfp_men = round((1.2*bmi)+(0.23*int(age))-16.2, 2)
        return bfp_men
    else:
        if age < 15:
            bfp_women = round((1.51*bmi)+(0.7*int(age))+1.4, 2)
        elif age >= 15:
            bfp_women = round((1.2*bmi)+(0.23*int(age))-5.4, 2)
        return bfp_women


# 公式二

In [ ]:
def bmi_formula_b(gender, age, height, weight):
    kg = round(float(weight)*0.45359237, 3)
    meters = round((float(height)*2.54) / 100, 3)
    bmi = round(kg / pow(meters, 2), 1)
    # print("公式二\nage:", age, "weights:", kg, "height:", meters, "m\nbmi:", bmi)
    if gender == "m":
        bfp_men = round((1.39*bmi)+(0.16*int(age))-19.34, 2)
        return bfp_men
    else:
        bfp_women = round((1.39*bmi)+(0.16*int(age))-9, 2)
    # elif gender == f:
        return bfp_women


# 公式三

原本海軍公式有錯，故改參考其他網站的正確換算公式

http://hugolohas.blogspot.com/2012/09/blog-post_5899.html

In [ ]:
import math
def bmi_formula_c(gender, height, weight, waist, neck, hip):
    # kg = round(float(weight)*0.45359237, 3)
    # height_m = round((float(height)*2.54) / 100, 3)
    height_cm = float(height)*2.54
    waist = float(waist)
    hip = float(hip)
    neck = float(neck)
    total_weight = float(weight)*1.082 + 94.42
    # print("height:", meters, "m\nwaist:", waist, " hip:", hip, " neck:", neck)
    # print(waist, waist/2.54*4.15)
    if gender == "m":
        # 以英吋測量：體脂肪％＝[86.01 x LOG10( 腰圍- 頸圍)] - [70.041 x LOG10(身高)] + 36.76
        # 以公分測量：體脂肪％＝495 / {1.0324 - 0.19077 x [LOG10(腰圍- 頸圍)] + 0.15456 x [LOG10(身高)]} - 450
        bfp_men = 495 / (1.0324 - (0.19077 * math.log10(waist - neck)) + (0.15456 * math.log10(height_cm))) - 450
        # bfp_men = ((total_weight - (total_weight - (waist*0.393700787*4.15)))/total_weight)*100
        return bfp_men
    else:
        # 以英吋測量：體脂肪％＝[163.205 x LOG10(腰圍 + 臀圍 - 頸圍)] - [97.684 x LOG10(身高)] - 78.387
        # 以公分測量：體脂肪％＝495 / {1.29579 - 0.35004 x [LOG10(腰圍 + 臀圍 - 頸圍)] + 0.22100 x [LOG10(身高)]} - 450
        bfp_women = 495 / (1.29579 - (0.35004 * math.log10(waist + hip - neck)) + (0.22100 * math.log10(height_cm))) - 450
        return bfp_women

# 從github找到的水下秤重資料集 bodyfat.csv
計算此252筆水下秤重量測的體脂率與上面3種公式計算出的體脂率 差異有多少

公式 1: 1776  平均誤差: 7.05  \
公式 2: 1543  平均誤差: 6.12  \
公式 3: 1003  平均誤差: 3.98 

比較平均誤差，第3個U.S. Navy方法，平均誤差3.98 \
為三種體脂率公式中，與水下秤重體脂率誤差最小的公式


In [ ]:
import pandas as pd
fn = "drive/MyDrive/body_shape/BodyFat.csv"
bodyfat = pd.read_csv(fn, encoding="utf-8", usecols=[1,3,4,5,6,7,8,9,10])
df = pd.DataFrame(bodyfat)
print(len(bodyfat))
print(bodyfat)
sum_fn1, sum_fn2, sum_fn3 = 0, 0, 0
for row_index, row in df.iterrows():
    bfp1 = bmi_formula_a("m", row.AGE, row.HEIGHT, row.WEIGHT)
    fn1_err = round(abs(bfp1 - row.BODYFAT), 3)
    sum_fn1 += fn1_err

    bfp2 = bmi_formula_b("m", row.AGE, row.HEIGHT, row.WEIGHT)
    fn2_err = round(abs(bfp2 - row.BODYFAT), 3)
    sum_fn2 += fn2_err

    bfp3 = bmi_formula_c("m", row.HEIGHT, row.WEIGHT, row.ABDOMEN, row.NECK, row.HIP)
    fn3_err = round(abs(bfp3 - row.BODYFAT), 3)
    sum_fn3 += fn3_err

    print("第", row_index+1, "筆")
    # print("女：", bfp_women)
    print("公式Ａ")
    print("男：", bfp1, "誤差：", fn1_err)
    print("公式Ｂ")
    print("男：", bfp2, "誤差：", fn2_err)
    print("US Navy - 公式C")
    print("男：", bfp3, "誤差：", fn3_err)
    # Adioposity (bmi) 肥胖指數
    print("公式1 誤差：", fn1_err, "公式2 誤差：", fn2_err, "公式3 誤差：", fn3_err)
    print("true bfp:", row.BODYFAT, " true BMI:", row.ADIPOSITY, "\n","_"*50)

# min_err_fn = min({"公式 1":sum_fn1, "公式 2":sum_fn2, "公式 3":sum_fn3})
min_err_fn = min(sum_fn1, sum_fn2, sum_fn3)
# {sum_fn1:"公式 1", sum_fn2:"公式 2", sum_fn3:"公式 3"}
# "總誤差最小的公式：", ,
print("="*100)
print("總誤差值：", min_err_fn)
print("公式 1:", round(sum_fn1), "公式 2:", round(sum_fn2), "公式 3:", round(sum_fn3))
print("公式 1平均誤差:", round( sum_fn1 / len(bodyfat), 2), "公式 2平均誤差:", round( sum_fn2 / len(bodyfat), 2), "公式 3平均誤差:", round( sum_fn3 / len(bodyfat), 2) )


In [ ]:
import math
#m	w	35	65.5	166.4	125.75	57.0		35.75	90.8	29.5	75	35.1	89.2							1.9
gender = "m"
height_cm = 166.4
weight = 57
waist = 75
hip = 89.2
neck = 34
# total_weight = float(weight)*1.082 + 94.42
# print("height:", meters, "m\nwaist:", waist, " hip:", hip, " neck:", neck)
# print(waist, waist/2.54*4.15)
if gender == "m":
    # 以英吋測量：體脂肪％＝[86.01 x LOG10( 腰圍- 頸圍)] - [70.041 x LOG10(身高)] + 36.76
    # 以公分測量：體脂肪％＝495 / {1.0324 - 0.19077 x [LOG10(腰圍- 頸圍)] + 0.15456 x [LOG10(身高)]} - 450
    bfp_men = 495 / (1.0324 - (0.19077 * math.log10(waist - neck)) + (0.15456 * math.log10(height_cm))) - 450
    # bfp_men = ((total_weight - (total_weight - (waist*0.393700787*4.15)))/total_weight)*100
    print(bfp_men)
else:
    # 以英吋測量：體脂肪％＝[163.205 x LOG10(腰圍 + 臀圍 - 頸圍)] - [97.684 x LOG10(身高)] - 78.387
    # 以公分測量：體脂肪％＝495 / {1.29579 - 0.35004 x [LOG10(腰圍 + 臀圍 - 頸圍)] + 0.22100 x [LOG10(身高)]} - 450
    bfp_women = 495 / (1.29579 - (0.35004 * math.log10(waist + hip - neck)) + (0.22100 * math.log10(height_cm))) - 450
    print(bfp_women)